In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py

train_data_path = 'data/train/'
test_data_path = 'data/test/'

def save_bottleneck_features(MODEL, image_size, module_name, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory(train_data_path, image_size, shuffle=False, batch_size=16)
    test_generator = gen.flow_from_directory(test_data_path, image_size, shuffle=False, batch_size=16, class_mode=None)

    train = model.predict_generator(train_generator)
    test = model.predict_generator(test_generator)
    with h5py.File("bottleneck_features/bottleneck_v3.h5") as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

#save_bottleneck_features(ResNet50, (224, 224), 'ResNet50')
#save_bottleneck_features(InceptionV3, (299, 299), 'InceptionV3', inception_v3.preprocess_input)
#save_bottleneck_features(Xception, (299, 299), 'Xception', xception.preprocess_input)

C:\Users\EWWIQQU\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
import h5py
import numpy as np
from sklearn.utils import shuffle
from keras.models import *
from keras.layers import *

X_train = []
X_test = []

#for filename in ["bottleneck_features/gap_ResNet50.h5", "bottleneck_features/gap_Xception.h5", "bottleneck_features/gap_InceptionV3.h5"]:
for filename in ["bottleneck_features/gap_ResNet50.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

print(X_train.shape[1:])

input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2)

(2048,)
Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 2s 99us/step - loss: 0.1624 - acc: 0.9344 - val_loss: 0.0868 - val_acc: 0.9664
Epoch 2/8
20000/20000 [==============================] - 2s 76us/step - loss: 0.0839 - acc: 0.9670 - val_loss: 0.0822 - val_acc: 0.9654
Epoch 3/8
20000/20000 [==============================] - 2s 85us/step - loss: 0.0753 - acc: 0.9711 - val_loss: 0.1217 - val_acc: 0.9522
Epoch 4/8
20000/20000 [==============================] - 2s 94us/step - loss: 0.0737 - acc: 0.9723 - val_loss: 0.0919 - val_acc: 0.9636
Epoch 5/8
20000/20000 [==============================] - 2s 84us/step - loss: 0.0648 - acc: 0.9748 - val_loss: 0.1020 - val_acc: 0.9606
Epoch 6/8
20000/20000 [==============================] - 2s 85us/step - loss: 0.0667 - acc: 0.9750 - val_loss: 0.1619 - val_acc: 0.9374
Epoch 7/8
20000/20000 [==============================] - 2s 89us/step - loss: 0.0627 - acc: 0.9757 - val_loss: 0.0550 - val_acc

In [1]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("data/sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(test_data_path, (224, 224), shuffle=False, batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('\\')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('data/pred.csv', index=None)
df.head(10)

NameError: name 'model' is not defined